In [2]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [34]:
# Combining and Merging Data Sets
df1 = DataFrame({'key1': ['b', 'b', 'a'], 'key2': ['one', 'two', 'one'], 'data1': range(3)})
df2 = DataFrame({'key1': ['a', 'b', 'd'], 'key2': ['one', 'two', 'one'], 'data2': range(3)})
df3 = DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df4 = DataFrame({'rkey': ['a', 'b', 'd'], 'data2': range(3)})
df5 = DataFrame({'lkey1': ['foo', 'foo', 'bar'], 'lkey2': ['one', 'two', 'one'], 'lval': [1,2,3]})
df6 = DataFrame({'rkey1': ['foo', 'foo', 'bar', 'bar'], 'rkey2': ['one', 'one', 'two', 'one'], 'lval': [4,5,6,7]})
df7 = DataFrame({'group_val': [3.5, 7]}, index = ['a', 'b'])
df8 = DataFrame(np.arange(8).reshape((4,2)), index = [['foo', 'foo', 'bar', 'bar'],['one', 'one', 'two', 'one']], columns = ['event1', 'event2'])
df9 = DataFrame(np.arange(6).reshape((3,2)), index = ['a', 'c', 'b'], columns = ['one', 'two'])

pd.merge(df1, df2, on = 'key1', suffixes = ('_left', '_right')) 
# suffixes option: specify strings to append to overlapping names in the left and right DataFrame objects
pd.merge(df3, df4, left_on = 'lkey', right_on = 'rkey')
pd.merge(df1, df2, how = 'outer')  # Default: inner; left, right, outer
pd.merge(df5, df6, left_on = ['lkey1', 'lkey2'], right_on = ['rkey1', 'rkey2'], how = 'outer')
pd.merge(df3, df7, left_on = 'lkey', right_index = True, how = 'outer')
pd.merge(df5, df8, left_on = ['lkey1', 'lkey2'], right_index = True)
# Hierarchically indexed data
# Other options: left_index, right_index, sort

,lkey1,lkey2,lval,event1,event2
0,foo,one,1,0,1
0,foo,one,1,2,3
2,bar,one,3,6,7


In [59]:
# Concatenating along an axis, p204
arr1 = np.arange(12).reshape((3,4))
print np.concatenate([arr1, arr1], axis = 1)

s1 = Series([0, 1], index=['a', 'b'])
s2 = Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = Series([5, 6], index=['f', 'g'])
s4 = pd.concat([s1*5, s3])

result = pd.concat([s1, s2, s3], keys= ['one', 'two', 'three'])  # create a hierarchical index on the concatenate axis
print result
print result.unstack()
print pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])  # In the case of combining Series along axis=1, the keys become the DataFrame column headers
print pd.concat([df9, df7], axis = 1, keys = ['level1', 'level2'], names = ['upper', 'lower'])
print pd.concat({'level1': df9, 'level2': df7}, axis = 1)   # If you pass a dict of objects instead of a list, the dict’s keys will be used for the keys option:
print pd.concat([s1, s4], axis = 1)
print pd.concat([s1, s4], axis = 1, join = 'inner')
print pd.concat([s1, s4], axis = 1, join_axes = [['a', 'c', 'b', 'e']])
print pd.concat([s1, s2], ignore_index = True)


[[ 0  1  2  3  0  1  2  3]
 [ 4  5  6  7  4  5  6  7]
 [ 8  9 10 11  8  9 10 11]]
one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64
         a    b    c    d    e    f    g
one    0.0  1.0  NaN  NaN  NaN  NaN  NaN
two    NaN  NaN  2.0  3.0  4.0  NaN  NaN
three  NaN  NaN  NaN  NaN  NaN  5.0  6.0
******************************
   one  two  three
a  0.0  NaN    NaN
b  1.0  NaN    NaN
c  NaN  2.0    NaN
d  NaN  3.0    NaN
e  NaN  4.0    NaN
f  NaN  NaN    5.0
g  NaN  NaN    6.0
upper level1        level2
lower    one two group_val
a          0   1       3.5
b          4   5       7.0
c          2   3       NaN
  level1        level2
     one two group_val
a      0   1       3.5
b      4   5       7.0
c      2   3       NaN
     0  1
a  0.0  0
b  1.0  5
f  NaN  5
g  NaN  6
   0  1
a  0  0
b  1  5
     0    1
a  0.0  0.0
c  NaN  NaN
b  1.0  5.0
e  NaN  NaN
0    0
1    1
2    2
3    3
4    4
dtype: int64


In [46]:
# Combining Data with Overlap, first get data from table1, if nan, get data from table2
a = Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan], index=['f', 'e', 'd', 'c', 'b', 'a'])
b = Series(np.arange(len(a), dtype=np.float64), index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
df1 = DataFrame({'a': [1., np.nan, 5., np.nan], 'b': [np.nan, 2., np.nan, 6.], 'c': range(2, 18, 4)})
df2 = DataFrame({'a': [5., 4., np.nan, 3., 7.], 'b': [np.nan, 3., 4., 6., 8.]})

np.where(pd.isnull(a), b, a)
a.combine_first(b)
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


In [67]:
# Reshaping and Pivoting
# Reshaping with Hierarchical Indexing
data = DataFrame(np.arange(6).reshape((2,3)), index = pd.Index(['Ohio', 'Colorado'], name = 'state'), columns = pd.Index(['one', 'two', 'three'], name = 'number'))
print data
result = data.stack()
print result.unstack(0)

s1 = Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd']) 
s2 = Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two']) 
print data2
print data2.unstack()
print data2.unstack().stack(dropna=False)
print '*'*30

df = DataFrame({'left': result, 'right': result + 5}, columns=pd.Index(['left', 'right'], name='side'))
print df
print df.unstack('state').stack('side').swaplevel('side', 'number').sortlevel('side')

number    one  two  three
state                    
Ohio        0    1      2
Colorado    3    4      5
state   Ohio  Colorado
number                
one        0         3
two        1         4
three      2         5
one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64
       a    b    c    d    e
one  0.0  1.0  2.0  3.0  NaN
two  NaN  NaN  4.0  5.0  6.0
one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64
******************************
side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10
state         Ohio  Colorado
side  number                
left  one        0         3
      two        1         4
      three      2         5
right one        5         8
     

In [75]:
# Pivoting “long” to “wide” Format
ldata = pd.read_csv('Workbook1.csv')
ldata['value2'] = np.random.randn(len(ldata))
pivot = ldata.pivot('date', 'item', 'value')
pivot = ldata.pivot('date', 'item')
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked

value                    value2                    
item      infl   realgdp unemp      infl   realgdp     unemp
date                                                        
12/31/59   NaN  2785.204   NaN       NaN -0.134104       NaN
3/31/59   0.00  2710.349   5.8  0.364424  1.309867  0.806347
6/30/59   2.34  2778.801   5.1 -1.282254  0.363119 -0.585538
9/30/59   2.74  2775.488   5.3 -1.140528 -0.855505 -1.625056

In [81]:
## Data transformation
# Removing Duplicates
data = DataFrame({'k1': ['one'] * 3 + ['two'] * 4, 'k2': [1, 1, 2, 3, 3, 4, 4], 'v1': range(7)})
data.duplicated()
data.drop_duplicates(['k1', 'k2'], keep='last')   # take_last = True

,k1,k2,v1
1,one,1,1
2,one,2,2
4,two,3,4
6,two,4,6


In [84]:
# Transforming Data Using a Function or Mapping
# Using map is a convenient way to perform element-wise transformations and other data cleaning-related operations.
data = DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami', 'corned beef', 'Bacon', 'pastrami', 'honey ham', 'nova lox'], 
                  'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
meat_to_animal = { 'bacon': 'pig', 'pulled pork': 'pig', 'pastrami': 'cow', 'corned beef': 'cow', 'honey ham': 'pig', 'nova lox': 'salmon'}
data['animal'] = data['food'].map(str.lower).map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [85]:
# Replace values
data = Series([1., -999., 2., -999., -1000., 3.])
data.replace(-999, np.nan)
data.replace([-999, -1000], np.nan)
data.replace([-999, -1000], [np.nan, 0])
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [94]:
# Renaming Axis Indexes
data = DataFrame(np.arange(12).reshape((3, 4)), index=['Ohio', 'Colorado', 'New York'], columns=['one', 'two', 'three', 'four'])
data.index = data.index.map(str.upper)
data.rename(index = str.title, columns = str.upper)
_ = data.rename(index = {'OHIO': 'INDIANA'}, columns = {'three': 'peekaboo'}, inplace = True)
data

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [4]:
# Discretization and Binning
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]
group_names= ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
data = np.random.rand(20)
cats = pd.cut(ages, bins, labels = group_names, right = False)
cats1 = pd.cut(data, 4, precision = 2)  # cut into equal length bins
catsq1 = pd.qcut(data, 4)   # cut into quartiles, equal sized
catsq = pd.qcut(data, [0, 0.1, 0.4, 0.75, 1])
cats
cats.codes
pd.value_counts(cats)
catsq.value_counts()
print '*'*30
print cats1.value_counts()

******************************
(0.026, 0.25]    3
(0.25, 0.48]     5
(0.48, 0.71]     5
(0.71, 0.93]     7
dtype: int64


In [121]:
# Detecting and Filtering Outlier
data = DataFrame(np.random.randn(1000, 4))
data.describe()
data[(np.abs(data)>3).any(1)]  # select all rows having a value > +-3
data[np.abs(data)>3] = np.sign(data) *3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.013178,0.029405,0.007056,-0.010801
std,0.967642,0.997839,0.958048,0.988979
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.639469,-0.610736,-0.661881,-0.658146
50%,-0.004637,0.039815,0.018655,-0.005824
75%,0.630381,0.752968,0.698894,0.644083
max,3.000000,2.930711,3.000000,2.935556


In [145]:
# Permutation and Random Sampling
df = DataFrame(np.arange(20).reshape(5,4))
sampler = np.random.permutation(5)
df.take(sampler)
df.ix[sampler]
df.take(np.random.permutation(len(df))[:3])  # to select a random subset without replacement
df.take(np.random.randint(0, len(df), size = 6))

,0,1,2,3
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15
3,12,13,14,15
0,0,1,2,3
0,0,1,2,3


In [167]:
# Computing indicator/ dummy variables
df = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 'data1': range(6)})
dummies = pd.get_dummies(df['key'], prefix = 'key')  # create dummy variables for qualitative variable
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

# movies = pd.read_csv('WorkBook1.csv')
genre_iter = (set(x.split('|')) for x in movies.genres)
genres = sorted(set.union(*genre_iter))
dummies = DataFrame(np.zeros((len(movies), len(genres))), columns = genres)
for i, gen in enumerate(movies.genres):
    dummies.ix[i, gen.split('|')] = 1
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.ix[1]
pd.get_dummies(pd.cut(ages, bins, labels = group_names))



,Youth,YoungAdult,MiddleAged,Senior
0,1,0,0,0
1,1,0,0,0
2,1,0,0,0
3,0,1,0,0
4,1,0,0,0
5,1,0,0,0
6,0,0,1,0
7,0,1,0,0
8,0,0,0,1
9,0,0,1,0


In [192]:
# String Manipulation, p207
val = 'a,b,  guido'
pieces = [x.strip() for x in val.split(',')]
print pieces
print '::'.join(pieces)
print 'guido' in val
print val.index(',')
print val.find(':')
print val.count(',')
print val.replace(',', '::')

['a', 'b', 'guido']
a::b::guido
True
1
-1
2
a::b::  guido


In [213]:
# Regular expressions
import re
text = "foo bar\t baz  \tqux"  # whitespace characters" \s+
re.split('\s+', text)

regex = re.compile('\s+')
regex.split(text)
regex.findall(text)

text = """Dave dave@google.com 
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = '[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'
regex = re.compile(pattern, flags = re.IGNORECASE)  #re.IGNORECASE makes the regex case-insensitive
regex.findall(text)
m = regex.search(text)   # search: returns only the first match
text[m.start():m.end()]
print regex.match(text)   # match: only matches at the beginning of the string
print regex.sub('REDACTED', text)

# segment each address into its 3 components, p226
pattern = '([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags = re.IGNORECASE)
regex.findall(text)
print regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text)

regex = re.compile(r"""
(?P<username>[A-Z0-9._%+-]+)
@
(?P<domain>[A-Z0-9.-]+)
\.
(?P<suffix>[A-Z]{2,4})""", flags=re.IGNORECASE|re.VERBOSE)
regex.match('wesm@bright.net').groupdict()

None
Dave REDACTED 
Steve REDACTED
Rob REDACTED
Ryan REDACTED

Dave Username: dave, Domain: google, Suffix: com 
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



{'domain': 'bright', 'suffix': 'net', 'username': 'wesm'}

In [239]:
# Vectorized string functions in pandas, p228
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com', 'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = Series(data)
data.str.contains('gmail')
pattern = '([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
data.str.findall(pattern, flags = re.IGNORECASE)
data.str[:5]


Dave     dave@
Rob      rob@g
Steve    steve
Wes        NaN
dtype: object